## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
import pprint, time
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\i353877\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(nltk_data, test_size=0.05)

### Build the vanilla Viterbi based POS tagger

In [4]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95794

In [5]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
print(tokens[:10])
len(tokens)

['Although', 'the', 'legislation', 'would', 'apply', 'to', 'acquisitions', 'involving', 'any', 'major']


95794

In [6]:
# vocabulary
V = set(tokens)
print(len(V))

12108


In [7]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
print(len(T))
print(T)

12
{'CONJ', 'NOUN', '.', 'PRON', 'ADV', 'DET', 'NUM', 'ADJ', 'ADP', 'X', 'VERB', 'PRT'}


In [8]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [9]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [10]:
#check if the word is present in train_set
def check_presence(word, train_bag = train_tagged_words):
    found = False
    for pair in train_bag:
        if pair[0] == word:
            found = True
            break
    
    return found

In [11]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [12]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [13]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [14]:
tags_df

,CONJ,NOUN,.,PRON,ADV,DET,NUM,ADJ,ADP,X,VERB,PRT
CONJ,0.000464,0.348632,0.035234,0.057024,0.053315,0.120538,0.040797,0.119147,0.051924,0.008345,0.159481,0.005100
NOUN,0.042690,0.264151,0.240184,0.004808,0.016974,0.013222,0.009616,0.012129,0.176951,0.028885,0.146281,0.044110
.,0.057337,0.222182,0.094069,0.065759,0.052410,0.172191,0.081616,0.044436,0.091292,0.027594,0.088694,0.002329
PRON,0.004971,0.210325,0.040535,0.008031,0.034034,0.009178,0.007266,0.073805,0.022945,0.094455,0.482218,0.012237
ADV,0.006974,0.030223,0.137164,0.014945,0.080372,0.068084,0.030887,0.129857,0.120558,0.021920,0.345732,0.013285
DET,0.000603,0.638731,0.017605,0.003738,0.012782,0.005306,0.022429,0.204269,0.009406,0.045942,0.038949,0.000241
NUM,0.014163,0.352611,0.116554,0.001475,0.002951,0.003246,0.184715,0.032753,0.034523,0.211272,0.018294,0.027442
ADJ,0.016930,0.702005,0.064596,0.000657,0.004438,0.005095,0.021039,0.065417,0.077416,0.020053,0.011834,0.010519
ADP,0.000853,0.320286,0.039889,0.069859,0.013332,0.323805,0.063033,0.107082,0.016852,0.035090,0.008426,0.001493
X,0.010483,0.060515,0.164231,0.055432,0.025254,0.055432,0.002700,0.017154,0.143107,0.074174,0.207274,0.184244


In [15]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [16]:
tagged_temp = Viterbi(['Android'])
print(tagged_temp)

[('Android', 'CONJ')]


In [17]:
# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

In [18]:
# tagging the test sentences
tagged_seq = Viterbi(test_tagged_words)

### Solve the problem of unknown words

In [19]:
##Solving the problem of unknown words
# Viterbi Heuristic
def modified_Viterbi(words, mod_option, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        found = check_presence(word)
        if (found):
            p = [] 
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]
                
                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p    
                p.append(state_probability)
            
            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
            state.append(state_max)
        else:
            #this means that the word was not present in the training dataset
            if(mod_option==1):
                #use rule based taggger
                state.append(rule_based_tagger([[word]]))
            elif(mod_option==2):
                #use Lexicon taggger
                state.append(unigram_Tagger([word]))
    return list(zip(words, state))

In [20]:
#Rule Based Tagger
# specify patterns for tagging
# example from the NLTK book
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*es$', 'VERB'),               # 3rd singular present
    (r'.*ould$', 'VERB'),              # modals
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                    # nouns
]

regexp_tagger = nltk.RegexpTagger(patterns)

def rule_based_tagger(word):
    res = regexp_tagger.tag_sents(word)
    return res[0][0][1]
    
#Lexicon(Unigram Tagger)
unigram_tagger = nltk.UnigramTagger(train_set, backoff=regexp_tagger)

def unigram_Tagger(word):
    res = unigram_tagger.tag(word)
    return res[0][1]

In [21]:
# tagging the test sentences using Rule Based Tagging modification
tagged_seq_mod1 = modified_Viterbi(test_tagged_words,1)

In [22]:
# tagging the test sentences using Laxicon Tagging modification
tagged_seq_mod2 = modified_Viterbi(test_tagged_words,2)

#### Evaluating tagging accuracy

In [23]:
# accuracy of Basic Viterbi Algorithm
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print(accuracy)

0.9094633346988938


In [24]:
##Accuracy of Modified Viterbi Algorithm with Rule Based Tagging modification
# accuracy
check = [i for i, j in zip(tagged_seq_mod1, test_run_base) if i == j] 
accuracy_mod1 = len(check)/len(tagged_seq_mod1)
print(accuracy_mod1)

0.9479721425645228


In [25]:
##Accuracy of Modified Viterbi Algorithm with Laxicon Tagging with Rule Based Tagging Backoff modification 
# accuracy
check = [i for i, j in zip(tagged_seq_mod2, test_run_base) if i == j] 
accuracy_mod2 = len(check)/len(tagged_seq_mod2)
print(accuracy_mod2)

0.9479721425645228


### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

We can clearly see that Plain Viterbi algorithm gives an accuracy of 90.9%, however there is a scope for improvement, as
the alogorithm is not able to perform well for words which were not there in the training data. Hence we need to put some 
backoff algorithm for such missing words scenario.

We tried below two versions of modification, 

Modification 1 : We tried to put an Rule Based Tagger for the scenarios where the words itself is not present in the 
                 training data. By using this approach we improved the accuracy to 94.8% from 90.9%
        
Modification 2 : For the second scenario, we have tried Unigram or Laxicon Tagger(backed off by Rule Based Tagger) for the 
                 scenario of missing word in train data. We had to use the back-off as Laxicon will also fail if the word is 
                 not present in the training data. By using this approach also we got an improved accuracy on plain Viterbi 
                 algorithm. The accuracy improved to 94.8% from 90.9%

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [26]:
## Testing

f = open("Test_sentences.txt", "r")
sentence_test = f.read()
words = word_tokenize(sentence_test)

#POS tagging using Plain Viterbi
tagged_seq_file = Viterbi(words)

#POS tagging using Viterbi with Rule Based Tagger
tagged_seq_file_mod1 = modified_Viterbi(words, 1)

#POS tagging using Viterbi with Laxicon Tagger backed-off by Rule Based Tagger
tagged_seq_file_mod2 = modified_Viterbi(words, 2)

In [27]:
type(tagged_seq_file)

list

In [28]:
length = len(tagged_seq_file)
length
tags_comp = []
temp_list = []

#Collecting all taggings
for i in range(0,length):
    temp_list = [tagged_seq_file[i][0], tagged_seq_file[i][1], tagged_seq_file_mod1[i][1], tagged_seq_file_mod2[i][1]]
    tags_comp.append(temp_list)

tags_comp

[['Android', 'CONJ', 'NOUN', 'NOUN'],
 ['is', 'VERB', 'VERB', 'VERB'],
 ['a', 'DET', 'DET', 'DET'],
 ['mobile', 'ADJ', 'ADJ', 'ADJ'],
 ['operating', 'NOUN', 'NOUN', 'NOUN'],
 ['system', 'NOUN', 'NOUN', 'NOUN'],
 ['developed', 'VERB', 'VERB', 'VERB'],
 ['by', 'ADP', 'ADP', 'ADP'],
 ['Google', 'CONJ', 'NOUN', 'NOUN'],
 ['.', '.', '.', '.'],
 ['Android', 'CONJ', 'NOUN', 'NOUN'],
 ['has', 'VERB', 'VERB', 'VERB'],
 ['been', 'VERB', 'VERB', 'VERB'],
 ['the', 'DET', 'DET', 'DET'],
 ['best-selling', 'ADJ', 'ADJ', 'ADJ'],
 ['OS', 'CONJ', 'NOUN', 'NOUN'],
 ['worldwide', 'CONJ', 'NOUN', 'NOUN'],
 ['on', 'ADP', 'ADP', 'ADP'],
 ['smartphones', 'CONJ', 'VERB', 'VERB'],
 ['since', 'ADP', 'ADP', 'ADP'],
 ['2011', 'CONJ', 'NUM', 'NUM'],
 ['and', 'CONJ', 'CONJ', 'CONJ'],
 ['on', 'ADP', 'ADP', 'ADP'],
 ['tablets', 'NOUN', 'NOUN', 'NOUN'],
 ['since', 'ADP', 'ADP', 'ADP'],
 ['2013', 'CONJ', 'NUM', 'NUM'],
 ['.', '.', '.', '.'],
 ['Google', 'CONJ', 'NOUN', 'NOUN'],
 ['and', 'CONJ', 'CONJ', 'CONJ'],
 ['Twitt

In [29]:
all_tags = pd.DataFrame(tags_comp, columns=['Word', 'Tagged by Plain Viterbi', 'Tagged by Modification 1', 'Tagged by Modification 2'])

In [30]:
all_tags

,Word,Tagged by Plain Viterbi,Tagged by Modification 1,Tagged by Modification 2
0,Android,CONJ,NOUN,NOUN
1,is,VERB,VERB,VERB
2,a,DET,DET,DET
3,mobile,ADJ,ADJ,ADJ
4,operating,NOUN,NOUN,NOUN
...,...,...,...,...
176,launch,NOUN,NOUN,NOUN
177,of,ADP,ADP,ADP
178,ICESAT-2,CONJ,NOUN,NOUN
179,Satellite,CONJ,NOUN,NOUN


In [31]:
#Method to prepare an indicator field for Modification by Rule Based Tagger
def changedInMod1(x):
    if(x['Tagged by Plain Viterbi'] != x['Tagged by Modification 1']):
        return 'Yes'
    else:
        return 'No'
    
#Method to prepare an indicator field for Modification by Rule Based Tagger
def changedInMod2(x):
    if(x['Tagged by Plain Viterbi'] != x['Tagged by Modification 2']):
        return 'Yes'
    else:
        return 'No'

In [32]:
all_tags['Changed in Modification1'] = all_tags.apply(changedInMod1, axis = 1)
all_tags['Changed in Modification2'] = all_tags.apply(changedInMod2, axis = 1)

In [33]:
all_tags

,Word,Tagged by Plain Viterbi,Tagged by Modification 1,Tagged by Modification 2,Changed in Modification1,Changed in Modification2
0,Android,CONJ,NOUN,NOUN,Yes,Yes
1,is,VERB,VERB,VERB,No,No
2,a,DET,DET,DET,No,No
3,mobile,ADJ,ADJ,ADJ,No,No
4,operating,NOUN,NOUN,NOUN,No,No
...,...,...,...,...,...,...
176,launch,NOUN,NOUN,NOUN,No,No
177,of,ADP,ADP,ADP,No,No
178,ICESAT-2,CONJ,NOUN,NOUN,Yes,Yes
179,Satellite,CONJ,NOUN,NOUN,Yes,Yes


### Evaluation of Rule Based Tagger Modification

In [35]:
#Get list of tags corrected by Modification 1(Rule Based Tagger)
listOfCorrectedTagsMod1 = all_tags[(all_tags['Changed in Modification1']=='Yes')]

In [36]:
listOfCorrectedTagsMod1[['Word','Tagged by Plain Viterbi','Tagged by Modification 1']]

,Word,Tagged by Plain Viterbi,Tagged by Modification 1
0,Android,CONJ,NOUN
8,Google,CONJ,NOUN
10,Android,CONJ,NOUN
15,OS,CONJ,NOUN
16,worldwide,CONJ,NOUN
18,smartphones,CONJ,VERB
20,2011,CONJ,NUM
25,2013,CONJ,NUM
27,Google,CONJ,NOUN
29,Twitter,CONJ,NOUN


In [37]:
#Number of corrected tags
print('No. of words in the file is ', len(all_tags))
print('No. of words corrected by Rule Based Tagger is ', len(listOfCorrectedTagsMod1))
print('Percentage of corrected tags by Rule Based Tagger is ', len(listOfCorrectedTagsMod1)/len(all_tags)*100)

No. of words in the file is  181
No. of words corrected by Rule Based Tagger is  37
Percentage of corrected tags by Rule Based Tagger is  20.441988950276244


### Evaluation of Laxicon Tagger(Backedoff by Rule Based Tagger) Modification

In [38]:
listOfCorrectedTagsMod2 = all_tags[(all_tags['Changed in Modification2']=='Yes')]

In [39]:
listOfCorrectedTagsMod2[['Word','Tagged by Plain Viterbi','Tagged by Modification 2']]

,Word,Tagged by Plain Viterbi,Tagged by Modification 2
0,Android,CONJ,NOUN
8,Google,CONJ,NOUN
10,Android,CONJ,NOUN
15,OS,CONJ,NOUN
16,worldwide,CONJ,NOUN
18,smartphones,CONJ,VERB
20,2011,CONJ,NUM
25,2013,CONJ,NUM
27,Google,CONJ,NOUN
29,Twitter,CONJ,NOUN


In [40]:
#Number of corrected tags
print('No. of words in the file is ', len(all_tags))
print('No. of words corrected by Laxicon Tagger(backed by Rule Based Tagger) is ', len(listOfCorrectedTagsMod2))
print('Percentage of corrected tags by Laxicon Tagger(backed by Rule Based Tagger)  is ', len(listOfCorrectedTagsMod2)/len(all_tags)*100)

No. of words in the file is  181
No. of words corrected by Laxicon Tagger(backed by Rule Based Tagger) is  37
Percentage of corrected tags by Laxicon Tagger(backed by Rule Based Tagger)  is  20.441988950276244


## Conclusion 

We can clearly see that some words like Android, Google, OS, 2011, 2013, 2015 etc were wrongly tagged as VERB which were 
corrected by both the modification techniques into NOUN, NUM as required

We can clearly see that 37 tags were corrected out of 181 tags i.e. we have corrected around 20.4% of all 181 tags from the given file.
This clearly indicates that we need multiple approaches on top of Plain Viterbi, specially to tackle the situation where a particular 
word was not present in the test set. This is a very common business scenario.